### Unstack Within Tables/Table Architecture
This shares an example of using Table object to transform parsed data into a pivoted, second table
JDL 5/22/25

In [1]:
import pandas as pd

def InstanceClassMockup(class_name):
    """
    Create blank, generic object for attribute assignment in test/demo scripts
    JDL 5/22/25
    """
    return type(class_name, (object,), {})()

# Example dataset with three key columns and one value column
data = {'date_wkstart': 2 * ['2025-04-01', '2025-04-08', '2025-04-15'],
    'pl_abbr': ['ProdA', 'ProdB', 'ProdA', 'ProdB', 'ProdA', 'ProdA'],
    'retailer': 3 * ['WMT'] + 3 * ['TGT'],
    'units_redeemed': [100, 200, 300, 400, 500, 600]}

# Mock Tables collection and individual Table instances
tbls = InstanceClassMockup('Tables')
tbls.ModelRaw, tbls.Model = InstanceClassMockup('Table'), InstanceClassMockup('Table')

# Set index list/col name attribute
tbls.ModelRaw.idx = ['date_wkstart', 'pl_abbr', 'retailer']

# Set raw data .df from example dataset
tbls.ModelRaw.df = pd.DataFrame(data)
print('\nRaw Data multiindex columns:', tbls.ModelRaw.idx, '\n')
tbls.ModelRaw.df


Raw Data multiindex columns: ['date_wkstart', 'pl_abbr', 'retailer'] 



,date_wkstart,pl_abbr,retailer,units_redeemed
0,2025-04-01,ProdA,WMT,100
1,2025-04-08,ProdB,WMT,200
2,2025-04-15,ProdA,WMT,300
3,2025-04-01,ProdB,TGT,400
4,2025-04-08,ProdA,TGT,500
5,2025-04-15,ProdA,TGT,600


In [ ]:
"""
*** Procedure to pivot raw data to new Table/DataFrame ***
Pandas unstack pivots a value column's data on the last index column in a multiindex while
maintaining data correspondence with the other multiindex columns. It names the
resulting columns with the values from pivot column and names the df columns the pivot column 
name. To keep clarity on the values, we rename unstacked columns to be original column 
name + the pivot column values suffix. We also delete the df columns name as superfluous.

JDL 5/22/25
"""
class Model:
    def UnstackRawDataProcedure(self, tbl_raw, tbl_final, col_values):
        """
        Pivot raw data to unstacked format
        """
        self.CopyAndSetIndex(tbl_raw, tbl_final)
        self.UnstackValuesAndSetColNames(tbl_raw, tbl_final, col_values)
        
    def CopyAndSetIndex(self, tbl_raw, tbl_final):
        """
        Set multiindex for raw data DataFrame
        """
        tbl_final.df = tbl_raw.df.copy().set_index(tbl_raw.idx)

    def UnstackValuesAndSetColNames(self, tbl_raw, tbl_final, col_values):
        """
        Unstack and set column names based on last key in index
        """
        tbl_final.df = tbl_final.df[col_values].unstack()
        unstack_key = tbl_raw.idx[-1]

        # Rename columns to keep continuity to col_values name
        dict_cols = {col: col_values + f'_{col}' for col in tbl_final.df.columns}
        tbl_final.df = tbl_final.df.rename(columns=dict_cols)
        
        # Remove columns name created by unstack
        tbl_final.df.columns.name = None

In [3]:
#Demo with individual methods
mdl = Model()
mdl.CopyAndSetIndex(tbls.ModelRaw, tbls.Model)
print('\nRaw data with multiindex:\n', tbls.Model.df)
mdl.UnstackValuesAndSetColNames(tbls.ModelRaw, tbls.Model, 'units_redeemed')
print('\nUnstacked data with new column names:\n', tbls.Model.df)


Raw data with multiindex:
                                units_redeemed
date_wkstart pl_abbr retailer                
2025-04-01   ProdA   WMT                  100
2025-04-08   ProdB   WMT                  200
2025-04-15   ProdA   WMT                  300
2025-04-01   ProdB   TGT                  400
2025-04-08   ProdA   TGT                  500
2025-04-15   ProdA   TGT                  600

Unstacked data with new column names:
                       units_redeemed_TGT  units_redeemed_WMT
date_wkstart pl_abbr                                        
2025-04-01   ProdA                   NaN               100.0
             ProdB                 400.0                 NaN
2025-04-08   ProdA                 500.0                 NaN
             ProdB                   NaN               200.0
2025-04-15   ProdA                 600.0               300.0


In [4]:
#Demo with Procedure call
mdl = Model()
mdl.UnstackRawDataProcedure(tbls.ModelRaw, tbls.Model, 'units_redeemed')
display(tbls.Model.df)

units_redeemed_TGT  units_redeemed_WMT
date_wkstart pl_abbr                                        
2025-04-01   ProdA                   NaN               100.0
             ProdB                 400.0                 NaN
2025-04-08   ProdA                 500.0                 NaN
             ProdB                   NaN               200.0
2025-04-15   ProdA                 600.0               300.0